In [2]:
import cv2
import numpy as np


def object_Detection_SIFT(image,image_ref):
    # grayscaling the images 
    image_1 = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image_2 = cv2.cvtColor(image_ref,cv2.COLOR_BGR2GRAY)
    #creating the object of SIFT class
    sift = cv2.SIFT()
    #finding the key and des
    key_1,des_1 = sift.detectAndCompute(image_1,None)
    key_2,des_2 = sift.detectAndCompute(image_2,None)
    #create the object of matcher 
    bf = cv2.BFMatcher(cv2.NORM_L2,crossCheck = False)
    #now matching the 2 dis
    match = bf.knnMatch(des_1 , des_2 , k=2)
    #sorting the match 
    #match = sorted(match, key = lambda x:x.distance)
    # Store good matches using Lowe's ratio test
    good_matches = []
    for m,n in match:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m) 

    return len(good_matches)
    #return len(match)
    
    
#its the refrence image 
ref_image = cv2.imread('images/box_in_scene.png')
#creating the object oof videocapture for using web cam
cap = cv2.VideoCapture(0)

while True:
    #read() function return a bool value and a image 
    ret,frame = cap.read()
    # Get height and width of webcam frame
    height, width = frame.shape[:2]

    # Define ROI Box Dimensions
    left_top_x = (width/2)-(width/4)
    left_top_y = (height/2)-(height/4)
    right_bottom_x = left_top_x+(width/2)
    right_bottom_y = left_top_y+(height/2)
    
    #draw rectangle of ur ROI Box Dimensions on image 
    cv2.rectangle(frame,(left_top_x,left_top_y),(right_bottom_x,right_bottom_y),(0,255,0),3)
    
    #crop the image in ROI Box Dimensions
    cropped = frame[left_top_y:right_bottom_y,left_top_x:right_bottom_x]

    # Flip frame orientation horizontally
    frame = cv2.flip(frame,1)
    #its the threshold for comparing if no. of match larger than threshold then only object detected
    threshold = 25
    #calling our object dection method 
    matches_ = object_Detection_SIFT(cropped,ref_image)
    no_of_matches = "No. of matches are "+str(matches_)
    cv2.putText(frame,no_of_matches,(left_top_x,right_bottom_y+27),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
    
    if matches_ > threshold :
        cv2.putText(frame,"Object Found",(left_top_x,left_top_y),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
    
    cv2.imshow('Object Detector', frame)
    cv2.imwrite('Object Detector using SIFT.jpg', frame)
    if cv2.waitKey(1)  ==13:
        break
    
cap.release()
cv2.destroyAllWindows()  